过滤法特征选择，按特征发散程度或与目标变量的之间的相关性对各个特征进行评分
单变量过滤方法：考虑单个特征与目标变量的相关性
多变量过滤方法：同时考虑特征变量和目标变量的相关关系和特征之间的相互关系

1. 方差阈值法（Variance Threshold)
是一种无监督特征选择方法

In [1]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold

In [3]:
# dataset
X = np.array([
    [1, 2, 3, 4],
    [1, 6, 7, 9],
    [1, 4, 4, 2],
    [1, 4, 6, 1],
    [0, 7, 3, 2],
    [1, 5, 2, 6]
])
# 设置阈值为1
selector = VarianceThreshold(1.0)
# train dataset
selector.fit(X)
# select feature
selected_X = selector.transform(X)
print("特征的方差: ", selector.variances_)
print("特征选择后的数据集:", selected_X)

特征的方差:  [0.13888889 2.55555556 3.13888889 7.66666667]
特征选择后的数据集: [[2 3 4]
 [6 7 9]
 [4 4 2]
 [4 6 1]
 [7 3 2]
 [5 2 6]]


2. 卡方统计量法（Chi-squared Statistic)-分类

In [5]:
from sklearn.feature_selection import chi2, SelectKBest

In [7]:
# dataset 
Y = np.array([1, 0, 1, 1, 1, 0])
selector = SelectKBest(chi2, k=2)
selector.fit(X, Y)
selected_X = selector.transform(X)

# 值越大对分类结果的贡献越大
print("特征卡方统计量值：", selector.scores_)
print("特征选择后的数据集：", selected_X)

特征卡方统计量值 [0.1        0.44642857 0.08       9.1875    ]
特征选择后的数据集 [[2 4]
 [6 9]
 [4 2]
 [4 1]
 [7 2]
 [5 6]]


3. 互信息法（Mutual Information) 
分类问题(mutual_info_classif) 当y为离散变量时
回归问题(mutual_info_regression) 当y为连续型变量时

In [9]:
from sklearn.feature_selection import mutual_info_classif

In [10]:
selector = SelectKBest(mutual_info_classif, k=2)
selector.fit(X, Y)
selected_X = selector.transform(X)

print("特征和目标变量的互信息值", selector.scores_)
print("特征选择的数据集", selected_X)

特征和目标变量的互信息值 [0.33888889 0.         0.         0.39444444]
特征选择的数据集 [[1 4]
 [1 9]
 [1 2]
 [1 1]
 [0 2]
 [1 6]]


4. F统计量法（F-score）
分类问题
回归问题

F值较大，表示变量之间具有较强的相关性

In [11]:
from sklearn.feature_selection import f_classif

In [13]:
selector = SelectKBest(f_classif, k=2)
selector.fit(X, Y)
selected_X = selector.transform(X)

print("特征F-统计值", selector.scores_)
print("特征选择后的数据值", selected_X)

特征F-统计值 [ 0.44444528  0.62892926  0.07207263 15.8918915 ]
特征选择后的数据值 [[2 4]
 [6 9]
 [4 2]
 [4 1]
 [7 2]
 [5 6]]


5. 皮尔逊相关系数（Pearson Correlation）
回归问题

用于衡量两个连续变量之间的线性相关性程度（两个随机变量协方差和标准差的商）

皮尔逊相关系数越大，表示特征与目标变量越相关

In [14]:
from scipy.stats import pearsonr
import pandas as pd

In [34]:
house_train_df = pd.read_csv("../data/california_house/train.csv", index_col=['id'])
Y = house_train_df['MedHouseVal']
X = house_train_df.drop(columns=['MedHouseVal'])

def ud_pearsonr(X, Y):
    result = np.array([pearsonr(x, Y) for x in X.T])
    return np.absolute(result[:, 0])

# 创建 SelectKBest 对象并拟合数据
selector = SelectKBest(ud_pearsonr, k=4)
selector.fit(X, Y)

selected_X = selector.transform(X)

print("Scores:\n", pd.Series(selector.scores_, index=X.columns))
print("Selected Features:", np.array(X.columns)[selector.get_support(indices=True)])

Scores:
 MedInc        0.701925
HouseAge      0.103210
AveRooms      0.366727
AveBedrms     0.067487
Population    0.038479
AveOccup      0.048475
Latitude      0.116499
Longitude     0.056742
dtype: float64
Selected Features: ['MedInc' 'HouseAge' 'AveRooms' 'Latitude']


多变量过滤方法

单变量过滤方法只考虑了单个特征变量和目标变量之间的相关性，无法避免特征之间的冗余。
多变量过滤方法则解决了这个问题。

6. 最大相关最小冗余（mRMR)
mRMR方法试图寻找一个与目标变量有较高相关度的特征子集，同时特征之间保持低的相关度，对相关度高的冗余特征进行惩罚。
mrmr_classif 处理分类问题
mrmr_regression 处理回归问题


In [38]:
from mrmr import mrmr_classif

In [39]:
X = np.array([
    [1, 2, 3, 4],
    [1, 6, 7, 9],
    [1, 4, 4, 2],
    [1, 4, 6, 1],
    [0, 7, 3, 2],
    [1, 5, 2, 6]
])
Y = np.array([1, 0, 1, 1, 1, 0])
X = pd.DataFrame(X, columns=['A','B','C','D'])

selected = mrmr_classif(X=X, y=Y, K=2)

print("选择的特征是: ", selected)

100%|██████████| 2/2 [00:00<00:00,  4.08it/s]

选择的特征是:  ['D', 'B']


综合比较

In [41]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, chi2, SelectKBest
from sklearn.feature_selection import mutual_info_classif, f_classif
from mrmr import mrmr_classif
from sklearn.tree import  DecisionTreeClassifier as DTC


In [48]:
# data prepare 
wine = load_wine()
wine
X = wine.data
Y = wine.target
# 特征子集大小
class_num = 5

In [49]:
# 方差阈值法
vt_sel = VarianceThreshold(1.0)
vt_sel.fit(X)
vt_trans_X = vt_sel.transform(X)
print("方差阈值法选择的特征：", vt_sel.get_support(True))

方差阈值法选择的特征： [ 1  3  4  9 12]


In [50]:
# 卡方统计量法
chi_sel = SelectKBest(chi2, k=class_num)
chi_sel.fit(X, Y)
chi_trans_X = chi_sel.transform(X)
print("卡方统计量选择的特征：", chi_sel.get_support(True))

卡方统计量选择的特征： [ 3  4  6  9 12]


In [52]:
# 互信息法
mi_sel = SelectKBest(mutual_info_classif, k=class_num)
mi_sel.fit(X, Y)
mi_trans_X = mi_sel.transform(X)
print("互信息法选择的特征是：", mi_sel.get_support(True))

互信息法选择的特征是： [ 0  6  9 11 12]


In [53]:
# F-统计量
F_sel = SelectKBest(f_classif, k = class_num)
F_sel.fit(X, Y)
F_trans_X = F_sel.transform(X)
print("F统计量选择的特征：", F_sel.get_support(True))

F统计量选择的特征： [ 0  6  9 11 12]


In [55]:
# mRMR方法
dfX = pd.DataFrame(X, columns=[i for i in range(len(wine.feature_names))])
F = mrmr_classif(dfX, Y, class_num)
mrmr_trans_X = X[:, F]
print("mRMR 选择的特征是：", np.sort(F).tolist())

100%|██████████| 5/5 [00:00<00:00, 22.42it/s]

mRMR 选择的特征是： [0, 6, 9, 11, 12]


In [56]:
# 定义模型计算精度
def ClassifyingModel(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=9)
    tree = DTC(criterion="entropy", max_depth=3, random_state=9)
    tree.fit(X_train, y_train)
    # 计算精度
    score = tree.score(X_test, y_test, sample_weight=None)
    return score


In [57]:
# 结果对比
print("方差阈值法：", ClassifyingModel(vt_trans_X, Y))
print("卡方统计量法：", ClassifyingModel(chi_trans_X, Y))
print("互信息法：", ClassifyingModel(mi_trans_X, Y))
print("F-统计量法: ", ClassifyingModel(F_trans_X, Y))
print("mRMR法: ", ClassifyingModel(mrmr_trans_X, Y))

方差阈值法： 0.8888888888888888
卡方统计量法： 0.9777777777777777
互信息法： 0.9777777777777777
F-统计量法:  0.9777777777777777
mRMR法:  0.9777777777777777
